In [ ]:
!pip install lumibot==2.9.13
!pip install alpaca-trade-api==3.1.1
!pip install torch
!pip install torchvision
!pip install torchaudio
!pip install transformers

In [ ]:
from lumibot.brockers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.stategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import timedelta

In [ ]:
API_KEY = ""
API_SECRET = ""
BASE_URL = ""

ALPACA_CREDS = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "Paper": True
}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from typing import Tuple
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)
labels = ["positive", "negative", "neutral"]

def estimate_sentiment(news):
    if news:
        tokens = tokenizer(news, return_tensors="pt", padding=True).to(device)

        result = model(tokens["input_ids"], attention_mask=tokens["attention_mask"])[
            "logits"
        ]
        result = torch.nn.functional.softmax(torch.sum(result, 0), dim=-1)
        probability = result[torch.argmax(result)]
        sentiment = labels[torch.argmax(result)]
        return probability, sentiment
    else:
        return 0, labels[-1]

In [ ]:
class AITrader(Strategy):
  def initialize(self,symbol:str="SPY",cash_at_risk:float=.5):
    self.symbol = symbol
    self.sleeptime = "24H"
    self.last_trade = None
    self.cash_at_risk = cash_at_risk
    self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

  def position_sizing(self):
    cash = self.get_cash()
    last_price = self.get_last_price(self.symbol)
    quantity = round(cash * self.cash_at_risk / last_price,0)
    return cash, last_price, quantity

  def get_dates(self):
    today = self.get_datetime()
    three_days_prior = today - Timedelta(days=3)
    return today.strftime('%Y-%,%m-%d'), three_days_prior.strftime('%Y-%,%m-%d')

  def get_sentiment(self):
    today, three_days_priotr = self.get_dates()
    news = self.api.get_news(symbol=self.symbol, start=three_days_priotr, end=today)
    news = [event.__dict__["raw"]["headline"] for event in news]

    probability, sentiment = estimate_sentiment(news)
    return probability, sentiment

  def on_trading_iteration(self):
    cash, last_price, quantity = self.position_sizing()
    probability, sentiment = self.get_sentiment()
    if cash > last_price:
      if sentiment == "positive" and probability > .999:
        if self.last_trade == "sell":
          self.sell_all()
        order = self.create_order(
            self.symbol,
            quantity,
            "buy",
            type="bracket",
            take_profit_pice = last_price*1.20,
            stop_loss_price=last_price*.95
        )
        self.submit_order(order)
        self.last_trade = "buy"

      elif sentiment == "negative" and probability > .999:
        if self.last_trade == "buy":
          self.sell_all()
        order = self.create_order(
            self.symbol,
            quantity,
            "sell",
            type="bracket",
            take_profit_pice = last_price*.8,
            stop_loss_price=last_price*1.05
        )
        self.submit_order(order)
        self.last_trade = "sell"


In [ ]:
start_date = datetime(2024,1,1)
end_date = datetime(2024,1,31)

broker = Alpaca(ALPACA_CREDS)
startegy = AITrader(name='AItradingBot', broker=broker,params={{"symbol":"SPY","cash_at_risk":.5}})
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    params={"symbol":"SPY","cash_at_risk":.5}
)

# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()